<a href="https://colab.research.google.com/github/JayThibs/Weak-Supervised-Learning-Case-Study/blob/main/text_classifier/notebooks/03_dbpedia_14_snorkel_dataset_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel --quiet
!pip install datasets --quiet
!pip install spacy --quiet

     |████████████████████████████████| 153kB 10.3MB/s 
     |████████████████████████████████| 3.8MB 8.7MB/s 
     |████████████████████████████████| 1.8MB 63.4MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 225kB 12.6MB/s 
     |████████████████████████████████| 245kB 26.7MB/s 
     |████████████████████████████████| 112kB 32.6MB/s 


In [2]:
import pandas as pd
from snorkel.labeling import labeling_function
from datasets import load_dataset
import re

In [3]:
dbpedia_dataset = load_dataset('dbpedia_14')

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


In [4]:
dbpedia_dataset = dbpedia_dataset.rename_column("label", "labels")

In [5]:
def merge_title_with_content(example):
    example["content"] = example["title"] + " " + example["content"]
    return example

In [6]:
dbpedia_dataset = dbpedia_dataset.map(merge_title_with_content, num_proc=10)

In [7]:
dbpedia_dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 70000
    })
})

In [8]:
data = {'labels': dbpedia_dataset['train']['labels'],
        'title': dbpedia_dataset['train']['title'],
        'content': dbpedia_dataset['train']['content']}

dbpedia_train_df = pd.DataFrame(data)
dbpedia_train_df['content'] = dbpedia_train_df['content'].str.lower()
dbpedia_train_df

,labels,title,content
0,0,E. D. Abbott Ltd,e. d. abbott ltd abbott of farnham e d abbott...
1,0,Schwan-Stabilo,schwan-stabilo schwan-stabilo is a german mak...
2,0,Q-workshop,q-workshop q-workshop is a polish company loc...
3,0,Marvell Software Solutions Israel,marvell software solutions israel marvell sof...
4,0,Bergan Mercy Medical Center,bergan mercy medical center bergan mercy medi...
...,...,...,...
559995,13,Barking in Essex,barking in essex barking in essex is a black ...
559996,13,Science & Spirit,science & spirit science & spirit is a discon...
559997,13,The Blithedale Romance,the blithedale romance the blithedale romance...
559998,13,Razadarit Ayedawbon,razadarit ayedawbon razadarit ayedawbon (burm...


In [9]:
data = {'labels': dbpedia_dataset['test']['labels'],
        'title': dbpedia_dataset['test']['title'],
        'content': dbpedia_dataset['test']['content']}

dbpedia_test_df = pd.DataFrame(data)
del data
dbpedia_test_df['content'] = dbpedia_test_df['content'].str.lower()
dbpedia_test_df

,labels,title,content
0,0,TY KU,ty ku ty ku /taɪkuː/ is an american alcoholic...
1,0,Odd Lot Entertainment,odd lot entertainment oddlot entertainment fo...
2,0,Henkel,henkel henkel ag & company kgaa operates worl...
3,0,GOAT Store,goat store the goat store (games of all type ...
4,0,RagWing Aircraft Designs,ragwing aircraft designs ragwing aircraft des...
...,...,...,...
69995,13,Energy Victory,energy victory energy victory: winning the wa...
69996,13,Bestiario,bestiario bestiario is a book of 8 short stor...
69997,13,Wuthering Heights,wuthering heights wuthering heights is a nove...
69998,13,L'Indépendant,l'indépendant l'indépendant is a newspaper pu...


## A gentle introduction to LFs

Labeling functions (LFs) help users encode domain knowledge and other supervision sources programmatically.

LFs are heuristics that take as input a data point and either assign a label to it (in this case, HAM or SPAM) or abstain (don’t assign any label). Labeling functions can be noisy: they don’t have perfect accuracy and don’t have to label every data point. Moreover, different labeling functions can overlap (label the same data point) and even conflict (assign different labels to the same data point). This is expected, and we demonstrate how we deal with this later.

Because their only requirement is that they map a data point a label (or abstain), they can wrap a wide variety of forms of supervision. Examples include, but are not limited to:

    Keyword searches: looking for specific words in a sentence
    Pattern matching: looking for specific syntactical patterns
    Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
    Distant supervision: using external knowledge base
    Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

### a) Exploring the training set for initial ideas

We’ll start by looking at 20 random data points from the train set to generate some ideas for LFs.


In [10]:
Abstain = -1
Company = 0
EducationalInstitution = 1
Artist = 2
Athlete = 3
OfficeHolder = 4
MeanOfTransportation = 5
Building = 6
NaturalPlace = 7
Village = 8
Animal = 9
Plant = 10
Album = 11
Film = 12
WrittenWork = 13

In [37]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

In [38]:
df_view = dbpedia_train_df[dbpedia_train_df['labels'] == 7].sample(100, random_state=42)
df_view

,labels,title,content
312823,7,Hedströmmen,hedströmmen hedströmmen is a river located in bergslagen in sweden.
296298,7,Gooseneck Glacier,gooseneck glacier gooseneck glacier is located in the fitzpatrick wilderness shoshone national forest in the us state of wyoming. the glacier is east of the continental divide in the northern wind river range and on the southeast flank of gannett peak the tallest mountain in wyoming. gooseneck glacier is separated from dinwoody glacier by a rocky outcropping and they are part of the largest grouping of glaciers in the american rocky mountains.
308505,7,Chiascio,chiascio the chiascio is a river of umbria central italy.it is part of the tiber basin.
286689,7,Lee (Vechte),lee (vechte) lee is a river of lower saxony germany.
306893,7,McClellan Creek,mcclellan creek mcclellan creek is a river in texas.named after george b. mcclellan who with his father-in-law randolph b. marcy made a survey of the area in 1851–52 looking for a route for the southern pacific railroad.
316572,7,Smoky Mountains (Idaho),smoky mountains (idaho) the smoky mountains are part of the rocky mountains and located on the west side of the wood river valley near sun valley idaho in the western united states. the range is within sawtooth national forest while part of it is within the sawtooth national recreation area.
292335,7,Maebongsan (Inje Gangwon-do),maebongsan (inje gangwon-do) maebongsan is a mountain in the county of inje gangwon-do in south korea. it has an elevation of 1271.1 m (4170 ft).
309591,7,Mount Colden,mount colden mount colden is the eleventh highest peak in the high peaks of the adirondack mountains new york usa.the peak is named after david s. colden an investor in the mcintyre iron works in 1836. the peak was briefly renamed mount mcmartin the next year but the older name persisted. the mountain is known for its distinctive trap dike a large crevice running up the center of the mountain which can clearly be seen from avalanche lake.there are two maintained trails up mount colden.
298948,7,Mount Angeles,mount angeles mount angeles is located just south of port angeles washington in the olympic national park. it is the highest peak in the hurricane ridge area. the summit which offers panoramic views of the strait of juan de fuca and many of the peaks of the interior olympic mountains can be climbed from the klahane ridge trail.the southeast side of mount angeles drains into morse creek thence into the strait of juan de fuca a couple of miles east of port angeles.
311067,7,Onatah Corona,onatah corona onatah corona is a feature on venus adjacent to ba'het corona. both features are surrounded by a ring of ridges and troughs which in places cut more radially-oriented fractures. the centers of the features also contain radial fractures as well as volcanic domes and flows. coronae are thought to form due to the upwelling of hot material from deep in the interior of venus.


In [39]:
# Showing the most common words to make the keyword creation process quicker
# We are choosing some of words in this list to populate the label:keyword dictionary

# from collections import Counter
# Counter(" ".join(df_view["content"]).split()).most_common(100) # slower than pd.Series()

# faster
pd.Series(' '.join(df_view.content).lower().split()).value_counts()[:100]

the          416
of           253
is           156
in           154
a            121
river        110
and           86
lake          48
mountain      38
it            37
to            33
on            30
creek         27
range         27
located       27
tributary     25
mountains     24
km)           19
its           19
south         19
miles         18
an            17
mount         17
part          17
from          17
are           16
southern      15
at            14
romania.      14
between       14
area          13
west          13
north         13
by            12
county        12
east          12
national      12
which         12
has           12
about         11
germany.      11
western       11
united        11
peak          10
san           10
as            10
flows         10
border         9
with           9
into           9
for            9
lies           9
southeast      8
within         8
near           8
de             8
m              8
central        8
m)            

## Creating Label:Keyword Dictionary

In [32]:
import spacy
from spacy.matcher import PhraseMatcher

# Creating keyword lists with string and split(', ').

keywords_list = ["""company, headquarter, corporate, finance, ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, inc., foundation, newspaper""".split(', '),
                 """university, students, bachelor, degree, school, academy, college, high, located, public, city, academy, county, institute, national, research, business, grammar, government, technology, medicine, mascot, academic, board, program, co-educational, economics, junior, science, schools, faculty""".split(', '),
                 """dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, english, american, chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, screenwriter, prose, poet, career, jazz, folk""".split(', '),
                 """rugby, player, football, professional, nfl, league, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, mlb, pitcher, fencer, driver, american, canadian, english, belgian, attended, season, champion, motorcycle""".split(', '),
                 """born, author, commentator, director, professor, leader, governor, politician, minister, president, died, representatives, assembly, republic, democratic, house of representatives, house, election, member of parliament, party, speaker, president, lawyer, liberal, candidate, election, deputy, prime minister, official""".split(', '),
                 """ship, aircraft, boeing, navy, destroyer, diesel, rail, warship, transport, submarine, trike, aerobatic, motorcar, railway, monoplane, vessel, motorcycle, navigation, railway, cars, airliner, naval, whaleship, rail, automobile, flown, ferry, wing, tailplane, car, convoy, flown, naval, locomotive, vehicle, flagship, cruise ship, boat, convoy, automaker, battleships""".split(', '),
                 """church, church, center, house, historic, dam, district, district, castle, hospital, institution, museum, victorian, farm, building, mall, restaurant, shopping mall, centre, supermarket, built in""".split(', '),
                 """river, mountain, hill, hills, land, lake, km, m, forest, creek, ocean, stream, strait, gulf, peak, elevation, glacier, volcanic, corona, tributary, mount, flows, border, watershed""".split(', '),
                 """rural, population, census, central, province, state, village, district, eastern, western, kilometres, km, mi, county, south, north, south-west, south-east, north-west, north-east, town, border, regional, capital, municipality, block, located, families, administrative, federation, india, united states, turkey, iran""".split(', '),
                 """habitat, species, beetles, subfamily, snail, endemic, family, extinct, shelled, squids, octopuses, wingspan, moth, frogs, dry land, humidity, fruit flies, genus, horse, racehorse, thoroughbred, farm, farms, frog, butterfly, class, flatworm, fish, bred, shark, tropical, subtropical, sphinx, dog, cat, mouse, lion, jaguar, sea, habitats, subclass, populations, fossil""".split(', '),
                 """plant, family, species, vegetative subspecies, dipterocarpaceae, tillandsia, genus, endemic, orchid, daisy, flower, flowering, plants, legume, habitat, green alga, lettuce, kelp, gutweed, succulent, microphylia, ulmus, coffee, soil, tropical, forest, wood, leaf, cultivated, tree, trees, aster, algae, sedge, grows, evergreen, fruit, seed, seeds, herbs, herb, bulrush, subtropical, violet, floral, meliaceae, wild, grass""".split(', '),
                 """album, country, singer, band, vocalist, member, guitarist, debut, studio, metal, records, produced, songs, live at, performance, indie, folk, musician, released, ep, music, rapper, official, cd, label, tracks, remastered, reissued, pop, release, recorded, producer, full-length, bonus, chart, reunited, grammy, billboard, featured, concert, singer-songwriter, songs""".split(', '),
                 """film, directed, starring, drama, based, released, novel, comedy, american, stars, produced, international, cinema, festival, documentary, biographical, love, romance, comedy-romance, movie""".split(', '),
                 """published, book, novel, journal, series, written, story, magazine, newspaper, daily, stories, peer-reviewed, fiction, covers, comic, volume, science, fantasy, edition, writer, law, created, research, established, history, weekly, issue, travel, academic, mystery, media, author, work, god""".split(', ')]

labels_list = ['Company', 'EducationalInstitution', 'Artist', 'Athlete', 'OfficeHolder',
               'MeanOfTransportation','Building', 'NaturalPlace', 'Village',
               'Animal', 'Plant', 'Album', 'Film', 'WrittenWork']

label_keyword_dict = dict(zip(labels_list, keywords_list))
print(label_keyword_dict['NaturalPlace'])

nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

['river', 'mountain', 'hill', 'hills', 'land', 'lake', 'km', 'm', 'forest', 'creek', 'ocean', 'stream', 'strait', 'gulf', 'peak', 'elevation', 'glacier', 'volcanic', 'corona', 'tributary', 'mount', 'flows', 'border', 'watershed']


In [33]:
# Running nlp.make_doc to speed things up
for label in label_keyword_dict:
  patterns = [nlp.make_doc(text) for text in label_keyword_dict[label]]
  matcher.add(label, patterns)

In [34]:
doc = nlp("monika bohge monika bohge (lüdenscheid 1947) is a german writer and actor.")
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(rule_id, span.text)

WrittenWork writer
Artist writer
Artist actor


In [40]:
matches[0]

(10780820017166675169, 11, 12)

In [41]:
itms = [item[0] for item in matches]



[10780820017166675169, 15148697031525903815, 15148697031525903815]

## Snorkel with Automated Labeling Functions

In [ ]:
# labeling function for all labels
@nlp_labeling_function() # labeling function for using spaCy
def lf_labeler(x, label_keyword_dict):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    for match_id in matches:
        labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
    if labels: # runs if labels is not empty
        return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

In [59]:
x = "monika bohge monika bohge (lüdenscheid 1947) is a german writer and actor."

In [89]:
class all_labeling_functions(object):
    def __init__(self, labels):
        for label in labels:
            setattr(self, label, functools.partial(self._myfunction, label))
    
    @nlp_labeling_function() # labeling function for using spaCy
    def lf_labeler(x, label_keyword_dict=label_keyword_dict):
        labels = []
        doc = nlp(x) # tokenizing the input text
        matches = PhraseMatcher(doc) # finding all the token words that match a specific label
        
        for match_id in matches:
          labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
        
        if len(labels) != 0: # runs if labels is not empty
          return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
        
        else:
          return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

    def _myfunction(self, label, request):
        print(request)
        return "app/data/mydata_%s.csv" % (label)

lfs = []

for i in range(len(labels_list)):
  lf_labeler = all_labeling_functions(i)
  # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

  lfs.append(all_labeling_functions(labels_list))

# myfunction = all_labeling_functions(labels_list)
# print(myfunction)

In [90]:
lfs

In [132]:
from types import FunctionType
from copy import copy

lf_list = []

@nlp_labeling_function() # labeling function for using spaCy
def lf_labeler(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    
    for match_id in matches:
      labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
    
    if len(labels) != 0: # runs if labels is not empty
      return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    
    else:
      return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

  # namespace = sys._getframe(0).f_globals
  # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

def copy_function(fn, name):

    return FunctionType(name=name)

for label in labels_list:
    name = 'lf_labeler_' + str(label)
    lf_list.append(copy_function(lf_labeler, name))

lf_list

TypeError: ignored

In [112]:
from snorkel.labeling.lf.nlp import nlp_labeling_function
from functools import partial
import sys

namespace = sys._getframe(0).f_globals

def all_labeling_functions(label): # labeling function for all labels

  @nlp_labeling_function() # labeling function for using spaCy
  def lf_labeler(x):
      labels = []
      doc = nlp(x) # tokenizing the input text
      matches = PhraseMatcher(doc) # finding all the token words that match a specific label
      
      for match_id in matches:
        labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
      
      if len(labels) != 0: # runs if labels is not empty
        return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
      
      else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

    # namespace = sys._getframe(0).f_globals
    # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

  return lf_labeler

print('lol')

lfs = []

# lf_labeler_list = {f'lf_labeler_{k}': partial(lf_labeler, i=k) for k in range(len(labels_list))}
# for i in range(len(labels_list)):
#   lfs.append(lf_labeler_list[f'lf_labeler_{i}'])


# lf_labeler_list = {f'lf_labeler_{k}': partial(lf_labeler, i=k) for k in range(len(labels_list))}
for label in labels_list:
  lfs[f'lf_labeler_{label}'] = all_labeling_functions(label)

SyntaxError: ignored

In [110]:
lf_labeler_list

{'lf_labeler_0': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_1': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_2': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_3': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_4': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_5': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_6': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_7': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []]}

In [106]:
lfs

[functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=0),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=1),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=2),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=3),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=4),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=5),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=6),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=7)]

In [ ]:
# from snorkel.labeling import LabelingFunction


# def keyword_lookup(x, keywords, label):
#     if any(word in x.text.lower() for word in keywords):
#         return label
#     return ABSTAIN


# def make_keyword_lf(keywords, label=SPAM):
#     return LabelingFunction(
#         name=f"keyword_{keywords[0]}",
#         f=keyword_lookup,
#         resources=dict(keywords=keywords, label=label),
#     )

## Snorkel with Manual Labeling Functions

In [ ]:
from snorkel.labeling.lf.nlp import nlp_labeling_function

@nlp_labeling_function()
def lf_company(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Company' for label_id in label_ids])
        return Company # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_educational_institution(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'EducationalInstitution' for label_id in label_ids])
        return EducationalInstitution # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_artist(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Artist' for label_id in label_ids])
        return Artist # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_athlete(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Athlete' for label_id in label_ids])
        return Athlete # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_office_holder(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'OfficeHolder' for label_id in label_ids])
        return OfficeHolder # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_mean_of_transportation(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'MeanOfTransportation' for label_id in label_ids])
        return MeanOfTransportation # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_building(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Building' for label_id in label_ids])
        return Building # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_natural_place(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'NaturalPlace' for label_id in label_ids])
        return NaturalPlace # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_village(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Village' for label_id in label_ids])
        return Village # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_animal(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Animal' for label_id in label_ids])
        return Animal # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_plant(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Plant' for label_id in label_ids])
        return Plant # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_album(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Album' for label_id in label_ids])
        return Album # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_film(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Film' for label_id in label_ids])
        return Film # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_written_work(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'WrittenWork' for label_id in label_ids])
        return WrittenWork # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

## Applying Snorkel Labeling Functions

In [75]:
company_dataset = dbpedia_train_df.iloc[0:1000]
company_dataset

,labels,title,content
0,0,E. D. Abbott Ltd,e. d. abbott ltd abbott of farnham e d abbott limited was a british coachbuilding business based in farnham surrey trading under that name from 1929. a major part of their output was under sub-contract to motor vehicle manufacturers. their business closed in 1972.
1,0,Schwan-Stabilo,schwan-stabilo schwan-stabilo is a german maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. it is the world's largest manufacturer of highlighter pens stabilo boss.
2,0,Q-workshop,q-workshop q-workshop is a polish company located in poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). they also run an online retail store and maintainan active forum community.q-workshop was established in 2001 by patryk strzelewicz – a student from poznań. initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablished.
3,0,Marvell Software Solutions Israel,marvell software solutions israel marvell software solutions israel known as radlan computer communications limited before 2007 is a wholly owned subsidiary of marvell technology group that specializes in local area network (lan) technologies.
4,0,Bergan Mercy Medical Center,bergan mercy medical center bergan mercy medical center is a hospital located in omaha nebraska. it is part of the alegent health system.
...,...,...,...
995,0,Paymenex,paymenex paymenex is a universal electronic payments technology that enables users to make electronic payment on the internet through an open membership and multichannel interchange network operated by a consortium of paymenex regional administrators worldwide and used by its members to deliver a range of electronic money products to their customers.
996,0,Buffalo Forge Company,buffalo forge company the buffalo forge company was formed in 1878 to manufacture blacksmith’s forges. their product offerings were expanded to include drilling machines in 1883 and steam engines and pumps in 1889.
997,0,ClickBoom,clickboom clickboom is a global design firm founded in canada in 1994.
998,0,Indie Recordings,indie recordings indie recordings is a norwegian record label focusing on hard rock and metal founded in 2005 by erik and espen røhne as a side project of indie distribution. in 2006 erlend gjerde joined indie recordings and indie distribution and together they developed the label from being a hobby project into one of the leading independent record labels in scandinavia releasing gold-record status albums with bands like kvelertak and satyricon.


In [78]:
from snorkel.labeling import PandasLFApplier
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=company_dataset)
L_test = applier.apply(df=company_dataset)

ValueError: ignored

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=14, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [ ]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

## Filtering out unlabeled data points

As we saw earlier, some of the data points in our train set received no labels from any of our LFs. These data points convey no supervision signal and tend to hurt performance, so we filter them out before training using a built-in utility.

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)